In [9]:
%%writefile example2.txt
first
second line
the third line
then a fourth line

Overwriting example2.txt


In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext()

In [10]:
sc.textFile('example.txt')

example.txt MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0

In [11]:
text_rdd = sc.textFile('example2.txt')

In [12]:
words = text_rdd.map(lambda line: line.split())

In [13]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [14]:
text_rdd.collect()

['first', 'second line', 'the third line', 'then a fourth line']

In [15]:
text_rdd.flatMap(lambda line: line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

In [16]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [17]:
services = sc.textFile('services.txt')

In [18]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [20]:
services.map(lambda line: line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [21]:
services.map(lambda line: line[1:] if line[0]=='#' else line).collect()

['EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [22]:
clean = services.map(lambda line: line[1:] if line[0]=='#' else line)

In [23]:
clean = clean.map(lambda line: line.split())

In [24]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [27]:
pairs = clean.map(lambda lst: (lst[3],lst[-1]))

In [28]:
pairs.collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [ ]:
# In order to use ReduceByKey methond, you need key-value pair rdd such as tuple to use it

In [29]:
rekey = pairs.reduceByKey(lambda amt1,amt2: amt1 + amt2)

In [30]:
rekey.collect()

[('State', 'Amount'),
 ('NY', '100.00750.00'),
 ('TX', '450.00200.00'),
 ('CA', '200.00500.00')]

In [31]:
rekey2 = pairs.reduceByKey(lambda amt1,amt2: float(amt1) + float(amt2))

In [32]:
rekey2.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [33]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [45]:
# Grab (State, Amount) as a Tuple
step1 = clean.map(lambda lst: (lst[3],lst[-1]))
# Reduce by Key
step2 = step1.reduceByKey(lambda amt1,amt2: float(amt1) + float(amt2))
# Get rid of (State, Amount) titles
step3 = step2.filter(lambda x: not x[0]=='State')
# Sort Results by Amount
step4 = step3.sortBy(lambda stAmount: stAmount[1],ascending=False)
# Action
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [42]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [47]:
# Grab (State, Amount) as a Tuple
step1 = clean.map(lambda lst: (lst[3],lst[-1]))

In [48]:
step1.collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [49]:
# Reduce by Key
step2 = step1.reduceByKey(lambda amt1,amt2: float(amt1) + float(amt2))

In [50]:
step2.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [51]:
# Get rid of (State, Amount) titles
step3 = step2.filter(lambda x: not x[0]=='State')

In [52]:
step3.collect()

[('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [53]:
# Sort Results by Amount
step4 = step3.sortBy(lambda stAmount: stAmount[1],ascending=False)

In [55]:
# Action
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [66]:
# Grab state and amounts
# Add them
# Get rid of ('State','Amount')
# Sort them by the amount value
clean.map(lambda lst: (lst[3],lst[-1]))\
.reduceByKey(lambda amt1,amt2 : float(amt1)+float(amt2))\
.filter(lambda x: not x[0]=='State')\
.sortBy(lambda stateAmount: stateAmount[1], ascending=False)\
.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [61]:
# Tuple Unpacking Example
x = ['ID','State', 'Amount']

In [62]:
def func1(lst):
    return lst[-1]

In [63]:
def func2(id_st_amt):
    #unpack values
    (Id,st,amt) = id_st_amt
    return amt

In [64]:
func1(x)

'Amount'

In [65]:
func2(x)

'Amount'

In [ ]:
# func2 is much more readable for later on